In [ ]:
import music21
import pathlib

from thesession.converter import ABCMusicConverter

In [ ]:
ABCMusicConverter("cooleys.abc", "cooleys").to_mp3(
    instrument="flute",
    tempo=190,
    max_notes=300,
    cut_silence=30,
    start=0.57,
    noise_amplitude=0.001,
    duration=120,
    vbr=8,
    clean_files=True
)

# CLAP

In [ ]:
import torch
from laion_clap import CLAP_Module

# Load pretrained CLAP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLAP_Module(enable_fusion=True) # For using fusion to manage >10s clips
model.load_ckpt()  # downloads pretrained weights
model = model.to(device)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [11]:
import torchaudio

waveform, sr = torchaudio.load("cooleys.mp3")

In [12]:
import torchaudio
import torch


def load_audio(filepath, target_sr=16000):
    waveform, sr = torchaudio.load(filepath)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.mean(dim=0).unsqueeze(0)  # Convert to mono, add batch dim

In [13]:
audio_tensor0 = load_audio("audio/1_cooleys/36371_0.mp3")
audio_tensor1 = load_audio("audio/1_cooleys/36371_1.mp3")
audio_tensor2 = load_audio("audio/8_the_banshee/32846_0.mp3")
audio_tensor3 = load_audio("audio/8_the_banshee/32846_1.mp3")

In [14]:
with torch.no_grad():
    embedding0 = model.get_audio_embedding_from_data(audio_tensor0, use_tensor=True)
    embedding1 = model.get_audio_embedding_from_data(audio_tensor1, use_tensor=True)
    embedding2 = model.get_audio_embedding_from_data(audio_tensor2, use_tensor=True)
    embedding3 = model.get_audio_embedding_from_data(audio_tensor3, use_tensor=True)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


In [15]:
embeddings = torch.cat([embedding0, embedding1, embedding2, embedding3], dim=0)
embeddings.shape

torch.Size([4, 512])

In [16]:
import torch.nn.functional as F

F.cosine_similarity(embeddings[None, :, :], embeddings[:, None, :], dim=-1)

tensor([[1.0000, 0.8502, 0.8558, 0.8473],
        [0.8502, 1.0000, 0.9414, 0.8412],
        [0.8558, 0.9414, 1.0000, 0.8524],
        [0.8473, 0.8412, 0.8524, 1.0000]], device='cuda:0')

# Dataset

In [3]:
import pathlib
import numpy as np
import pandas as pd

from thesession.dataset import TheSessionDataset

path = pathlib.Path("audio_flac")
tunes = [p.stem for p in path.iterdir() if p.is_dir()]
tunes.sort()

tunes_df = pd.DataFrame({"tune": tunes.copy(), "dataset": None})

prng = np.random.default_rng(42)
tunes = prng.permutation(tunes)

tunes_df.loc[tunes_df["tune"].isin(tunes[0 : int(0.15 * len(tunes))]), "dataset"] = "test"
tunes_df.loc[tunes_df["tune"].isin(tunes[int(0.15 * len(tunes)) : int(0.3 * len(tunes))]), "dataset"] = "validation"
tunes_df.loc[tunes_df["tune"].isin(tunes[int(0.3 * len(tunes)) :]), "dataset"] = "train"


tunes_df.to_csv("dataset.csv")

In [4]:
test_subset = tunes_df.loc[tunes_df["dataset"] == "test", "tune"]
val_subset = tunes_df.loc[tunes_df["dataset"] == "validation", "tune"]
train_subset = tunes_df.loc[tunes_df["dataset"] == "train", "tune"]

train_dataset = TheSessionDataset("audio_flac", subset=train_subset, sampling_rate=48000, fmt=".flac")
val_dataset = TheSessionDataset("audio_flac", subset=val_subset, sampling_rate=48000, fmt=".flac")
test_dataset = TheSessionDataset("audio_flac", subset=test_subset, sampling_rate=48000, fmt=".flac")

print("Training data", len(train_dataset))
print("Validation data", len(val_dataset))
print("Test data", len(test_dataset))

Training data 25418
Validation data 5430
Test data 5513


In [3]:
train_dataset[32]

(tensor([-0.0138, -0.0157, -0.0151,  ..., -0.0132, -0.0090, -0.0040]),
 tensor([ 0.0086,  0.0108,  0.0123,  ..., -0.0052, -0.0035, -0.0016]))

In [4]:
train_dataset[32][0].max()

tensor(0.0609)

In [5]:
train_dataset[32][0].min()

tensor(-0.0386)

In [6]:
%%timeit

train_dataset[0]

47 ms ± 3.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
import torch

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

In [8]:
import time

start = time.perf_counter()

for i, (x1, x2) in enumerate(train_loader):
    if i < 20:
        print(i, end="\r")
    else:
        break

end = time.perf_counter()
print("Duration: ", end - start)



Duration:  8.030320544996357


# Model

In [5]:
from thesession.model import TheSessionModel

model = TheSessionModel()
model.load_ckpt()

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [6]:
model.toggle_gradients(False)

Disabling gradient for parameter clap_model.model.logit_scale_a
Disabling gradient for parameter clap_model.model.logit_scale_t
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_real.weight
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_imag.weight
Disabling gradient for parameter clap_model.model.audio_branch.logmel_extractor.melW
Disabling gradient for parameter clap_model.model.audio_branch.bn0.weight
Disabling gradient for parameter clap_model.model.audio_branch.bn0.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.mel_conv

In [7]:
model.toggle_gradients(True, ['clap_model.model.audio_projection'])

Enabling gradient for parameter clap_model.model.audio_projection.0.weight
Enabling gradient for parameter clap_model.model.audio_projection.0.bias
Enabling gradient for parameter clap_model.model.audio_projection.2.weight
Enabling gradient for parameter clap_model.model.audio_projection.2.bias


# Entraînement

In [9]:
model.load("models/transfer_learning_audio_projection.pt")

In [1]:
import pandas as pd

from thesession.dataset import TheSessionDataset
from thesession.model import TheSessionModel
from thesession.training import eval_model, NTXentLoss

# Dataset
dataset = pd.read_csv("dataset.csv")

test_subset = dataset.loc[dataset["dataset"] == "test", "tune"]

test_dataset = TheSessionDataset(
    "audio_flac", subset=test_subset, sampling_rate=48000, fmt=".flac"
)

model = TheSessionModel(device="cuda")
model.load("models/transfer_learning_audio_projection.pt")


/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model.clap_model.model.audio_branch

HTSAT_Swin_Transformer(
  (spectrogram_extractor): Spectrogram(
    (stft): STFT(
      (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(480,), bias=False)
      (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(480,), bias=False)
    )
  )
  (logmel_extractor): LogmelFilterBank()
  (spec_augmenter): SpecAugmentation(
    (time_dropper): DropStripes()
    (freq_dropper): DropStripes()
  )
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (mel_conv2d): Conv2d(1, 96, kernel_size=(4, 12), stride=(4, 12))
    (fusion_model): AFF(
      (local_att): Sequential(
        (0): Conv2d(96, 24, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(24, 96, kernel_s

In [8]:
criterion = NTXentLoss()

eval_model(model, test_dataset, criterion, "cuda", num_workers=4)


/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


Duration of evaluation on test data: 94.86 seconds
Test Loss: 0.5987 | Test Accuracy: 0.8411 | 


(0.5986882602376056, 0.8411316648531012)

In [ ]:
model